In [1]:
!cp drive/My\ Drive/recsys.zip .

In [2]:
!unzip recsys.zip
!7z e members.csv.7z
!rm members.csv.7z
!7z e song_extra_info.csv.7z
!rm song_extra_info.csv.7z
!7z e songs.csv.7z
!rm songs.csv.7z
!7z e train.csv.7z
!rm train.csv.7z

Archive:  recsys.zip
  inflating: members.csv.7z          
  inflating: sample_submission.csv.7z  
  inflating: song_extra_info.csv.7z  
  inflating: songs.csv.7z            
  inflating: test.csv.7z             
  inflating: train.csv.7z            

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1349856 bytes (1319 KiB)

Extracting archive: members.csv.7z
--
Path = members.csv.7z
Type = 7z
Physical Size = 1349856
Headers Size = 130
Method = LZMA2:3m
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       2503827
Compressed: 1349856

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for 

In [49]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from tqdm.autonotebook import tqdm
from multiprocessing import cpu_count
from gensim.models import Word2Vec
from functools import partial
from sklearn.metrics import roc_auc_score

In [2]:
!pip install -q swifter
import swifter

In [66]:
train = pd.read_csv('train.csv')
songs = pd.read_csv('songs.csv')
songs_extra = pd.read_csv('song_extra_info.csv')

In [67]:
songs = pd.merge(songs, songs_extra, on='song_id', how='left')
songs.fillna('[unknown]', inplace=True)

Создадим для каждой песни новое имя - исполнитель: название песни

In [68]:
songs['artist_name'] = songs['artist_name'].swifter.apply(lambda x: x + ': ')
songs['new_name'] = songs['artist_name'] + songs['name']

In [69]:
songs = songs.set_index('song_id')
songs.drop(['song_length', 'genre_ids', 'composer', 'lyricist', 'language', 'isrc'], axis=1, inplace=True)

In [70]:
songs.loc['aRNy4rKgWyDwTjsMIcAbtpGvddDZ7L7OGUeJ09LAM2w=']

artist_name                    Alan Walker: 
name                        Faded (Restrung)
new_name       Alan Walker: Faded (Restrung)
Name: aRNy4rKgWyDwTjsMIcAbtpGvddDZ7L7OGUeJ09LAM2w=, dtype: object

Каждого юзера можно представить как среднее значение всех эмбеддингов песен, которые слушал этот юзер. Для этого нужно составить словарь юзер-песни

In [8]:
train = train[train['target'] == 1]
train = train[train['song_id'].isin(songs.index)]

In [35]:
%%time

user2songs = train.groupby('msno')['song_id'].apply(lambda arr: songs.loc[arr]['new_name'].tolist()).to_dict()

CPU times: user 25.3 s, sys: 85.4 ms, total: 25.4 s
Wall time: 25.3 s


In [37]:
user2songs['Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=']

['Various Artists: Lords of Cardboard',
 'Nas: Hip Hop Is Dead(Album Version (Edited))',
 'Soundway: Disco Africa',
 'BIGBANG TAEYANG: 眼| 鼻| 口 (Eyes| Nose| Lips)',
 '周湯豪 (NICKTHEREAL): 帥到分手',
 '李佳薇 (Jess Lee): 像天堂的懸崖',
 'A-Lin: 我能體諒',
 '韋禮安 (William Wei): 還是愛著你',
 '孫燕姿 (Yanzi Sun): 克卜勒 (Kepler)',
 '王詩安 (Diana Wang): 我不要再難過',
 '小宇-宋念宇: 說分手之後',
 '孫子涵 (Niko Sun): 回憶那麼傷 (Sad Memory)',
 '曹格 (Gary Chaw): 丑角',
 '周杰倫 (Jay Chou): 愛情廢柴',
 'Desiigner: Panda',
 "Drake: Hold On| We're Going Home",
 '陳星翰 (Starr Chen): 噹噹噹 (Dang Dang Dang)',
 'Nas: Made You Look',
 '大隸 (Da lead): 再給點時間 (Some more time)',
 '蔡健雅 (Tanya Chua): 空白格',
 '吳克群 (Kenji Wu): 越愛越難過',
 'Usher: Good Kisser',
 'Trey Songz: Na Na',
 '傅又宣 (Maggie Fu): 愛．這件事情',
 '林宥嘉 (Yoga Lin): 傻子',
 'Mike Will Made-It: 23',
 'G.E.M.鄧紫棋: 新的心跳 (Heartbeat)',
 '信樂團 (Shin Band): 回不去了',
 'G.E.M.鄧紫棋: 夜空中最亮的星',
 '蛋堡 (Soft Lipa): 煙霧瀰漫',
 '187 INC 謀殺有限公司: 鐘點情人 (Part time lover)',
 'Various Artists: 也可以 (Might as well)',
 '楊丞琳 (Rainie Yang): 單',
 '陳奕迅 (Eason C

In [71]:
songs = songs.drop(['artist_name', 'name'], axis=1).to_dict()['new_name']

In [41]:
workers = cpu_count() - 1
word_model = Word2Vec(
    size=16,
    min_count=10,
    window=2,
    negative=10,
    sg=1,
    workers=workers
)
sentences = list(user2songs.values())

In [42]:
word_model.build_vocab(sentences)

In [43]:
word_model.train(sentences, 
                 total_examples=word_model.corpus_count, 
                 epochs=20)

(66880362, 74292200)

## Similars

In [44]:
word_model.init_sims(replace=True)

In [45]:
def get_similars(name, model=word_model, k=10):
    return model.wv.similar_by_word(name, topn=k)

In [46]:
get_similars('Eminem: Berzerk')

[('Eminem: Stan', 0.9487439393997192),
 ('Flo Rida: Who Dat Girl', 0.9373338222503662),
 ('Eminem: Survival', 0.9304335117340088),
 ('Eminem: Lose Yourself', 0.9230690598487854),
 ('Eminem: Like Toy Soldiers', 0.9204150438308716),
 ('Eminem: Headlights', 0.9125577211380005),
 ("Eminem: When I'm Gone", 0.912090539932251),
 ('Will.I.Am: It’s My Birthday', 0.9120147228240967),
 ('Taio Cruz: Higher', 0.910914421081543),
 ('Jay-Z: Empire State Of Mind', 0.9108501672744751)]

Eminem предсказывается отлично, что неудивительно, так как он очень популярен. Посмотрим что-нибудь еще.

In [61]:
get_similars('Michael Jackson: Billie Jean')

[('Michael Jackson: The Way You Make Me Feel', 0.9641506671905518),
 ('Michael Jackson: Dangerous', 0.9613463878631592),
 ('Michael Jackson: Beat It', 0.9486440420150757),
 ('Michael Jackson: Smooth Criminal', 0.9470428228378296),
 ('Michael Jackson: Jam', 0.9438972473144531),
 ('ABBA: Gimme! Gimme! Gimme! (A Man After Midnight)', 0.9428439140319824),
 ('Donna Summer: Hot Stuff', 0.9324776530265808),
 ('Michael Jackson: Smooth Criminal (Radio Edit)', 0.9300098419189453),
 ('Michael Jackson: Black Or White', 0.9289754033088684),
 ('Backstreet Boys: Shape of My Heart', 0.9248590469360352)]

Тут все еще более очевидно и хорошо :) Идем дальше

In [73]:
songs

{'CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=': '張信哲 (Jeff Chang): 焚情',
 'o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=': 'BLACKPINK: PLAYING WITH FIRE',
 'DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=': 'SUPER JUNIOR: SORRY| SORRY',
 'dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=': 'S.H.E: 愛我的資格',
 'W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=': '貴族精選: Mary Had a Little Lamb',
 'kKJ2JNU5h8rphyW21ovC+RZU+yEHPM+3w85J37p7vEQ=': '貴族精選: となりのトトロ',
 'N9vbanw7BSMoUgdfJlgX1aZPE1XZg8OS1wf88AQEcMc=': '伍佰 & China Blue: 夢醒時分',
 'GsCpr618xfveHYJdo+E5SybrpR906tsjLMeKyrCNw8s=': '光良 (Michael Wong): 記得我愛你',
 'oTi7oINPX+rxoGp+3O6llSltQTl80jDqHoULfRoLcG4=': '林俊傑 (JJ Lin): 裂縫中的陽光 (Before Sunrise)',
 'btcG03OHY3GNKWccPP0auvtSbhxog/kllIIOx5grE/k=': 'Kodaline: The One',
 'HulM/OaHgD5kUyjNQjDUf8VZdsy7h4EJUIff79Cifwo=': 'D.L 羅時豐 (Daniel Lo): 紅花',
 'wypPzqFNdUJAqyBVxmFGaK4z7krUNWr5YqA0q0wi9eE=': '白安 (Ann): 對與錯',
 'fAZLdfQaLG76a6Ei4alt1eSjBM9rshQkiQEC6+n+y08=': 'Littlesong: I dreamt I dwelt In Marble halls| From T

In [72]:
songs['W0W4F9XsUbpz39KT4qqZCwFfK5pHJXJ1eo9LYKptHRM=']

'Xiu Xiu: Get Up'

In [74]:
get_similars('Xiu Xiu: Get Up')

KeyError: ignored

## Scores

Посчитаем эмбеддинги юзеров как среднее эмбеддингов их песен.

In [75]:
user_embeddings = defaultdict(partial(np.zeros, 16))
for user in tqdm(user2songs):
    k = 0
    for song in user2songs[user]:
        try:
            user_embeddings[user] = user_embeddings[user] + word_model.wv.word_vec(song)
            k += 1
        except:
            continue
    user_embeddings[user] = user_embeddings[user] / max(k, 1)

In [78]:
def get_user_embedding(user):
    return user_embeddings[user]

def get_song_embedding(song_id, model=word_model):
    try:
        return model.wv.word_vec(songs[song_id]) 
    except:
        return np.zeros(16)

Получим эмбеддинги для всех юзеров и всех песен.

In [79]:
song_embeddings = np.vstack(
                      train['song_id'].swifter.apply(get_song_embedding)
)
user_embeddings = np.vstack(
                      train['msno'].swifter.apply(get_user_embedding)
)

Простейшая рекомендательная система:

In [80]:
scores = (song_embeddings * user_embeddings).mean(axis=1)

In [81]:
roc_auc_score(train.target, scores)

0.6406722324901205

Catboost из прошлого задания выдал 0.755948. Разница довольно значительная. Но если нужен очень быстрый прототип рекомендательной системы или только симилары, то word2vec вполне неплохой вариант.